# Coordinates

This file contains the processing of coordinates data in the yelp restaurant dataset.

**Goal:** Provide a coordinates_processing function, which can be used in main.ipynb.

## 1. Get the coordinates dataset

The coordinates dataset only contains price, rating, review_count, name, id, coordinates (latitude, longitude) columns. Extraction of this dataset is shown in main.ipynb.

In [3]:
import pickle
import json
import os
import numpy as np
import pandas as pd
from math import radians, cos, sin, asin, sqrt

In [4]:
origin_data_file = '../Dataset/coordinates_data'

with open(origin_data_file, 'rb') as inFile:
    origin_data = pickle.load(inFile)

display(origin_data.head())
print("origin_data.shape: {0}".format(origin_data.shape))

,id,name,latitude,longitude,rating,review_count,price
0,bwCj2AcoOroZfCTxb6rCcg,A Better Burger,36.689639,-83.108766,3.5,6,2
1,S9S9kFJSkmfpbjFForCWLQ,El Castillo,36.726337,-83.099858,4.0,2,1
2,np8uV1xll22Yr-Q-B-ImkA,Rooster's Pub,36.758436,-83.027057,4.5,4,1
3,HGY1ojoLu07P_ky2LeRguQ,Redstone Restaurant,36.689259,-82.753040,4.5,3,1
4,J5XS3VmxnLKhNlpiwDJ-3A,Little Mexico,36.859367,-82.756744,4.0,5,1


origin_data.shape: (5119, 7)


## 2. Get census tract data
Extract cencus tract information in Virginia for further processing.

**Census tract geographic data**: The census tract data is downloaded from [2010 Census Gazetteer Files](https://www.census.gov/geo/maps-data/data/gazetteer2010.html).

In [25]:
tract_file = "../dataset/Gaz_tracts_national.txt"

with open(tract_file, 'r') as infile:
    tract_df = pd.read_csv(infile, sep = "\t")

tract_df = tract_df.loc[tract_df['USPS'] == 'VA'].reset_index(drop=True)

tract_df.columns = tract_df.columns.str.lower()

In [26]:
tract_df.head()

,usps,geoid,pop10,hu10,aland,awater,aland_sqmi,awater_sqmi,intptlat,intptlong
0,VA,51001090100,2941,4517,18529985,37927447,7.154,14.644,37.945138,-75.362044
1,VA,51001090200,6156,4259,187805809,36543634,72.512,14.110,37.953865,-75.503313
2,VA,51001090300,2335,1221,128431098,61781712,49.588,23.854,37.887621,-75.662907
3,VA,51001090400,6234,2687,193719276,35803197,74.795,13.824,37.752376,-75.598061
4,VA,51001090500,2849,1361,65940306,39496129,25.460,15.250,37.814134,-75.696946


In [27]:
tract_df.columns

Index(['usps', 'geoid', 'pop10', 'hu10', 'aland', 'awater', 'aland_sqmi',
       'awater_sqmi', 'intptlat',
       'intptlong                                                     '],
      dtype='object')

**Cencus tract data**: including sex by age, household income and age by language. These data is downloaded from [Census Reporter](https://censusreporter.org/profiles/04000US51-virginia/).

- **Sex by age data processing**

In [7]:
sex_by_age_path = '../dataset/sex_by_age/'

# Process data.csv
with open(os.path.join(sex_by_age_path, 'data.csv'), 'r') as infile:
    sex_by_age_df = pd.read_csv(infile)

# Drop US and Virginia rows
sex_by_age_df.drop(index=[0, 1], inplace=True)
sex_by_age_df.reset_index(inplace=True, drop=True)

# Get column names from metadata.json
with open(os.path.join(sex_by_age_path, 'metadata.json'), 'r') as infile:
    metadata = json.load(infile)

isMaleNow = False
change_cols = {}
for col in metadata['tables']['B01001']['columns'].items():
    if(col[1]['name'] == 'Total:'):
        change_cols[col[0]] = 'total_population'
    elif(col[1]['name'] == 'Male:'):
        change_cols[col[0]] = 'total_male_population'
        isMaleNow = True
    elif(col[1]['name'] == 'Female:'):
        change_cols[col[0]] = 'total_female_population'
        isMaleNow = False
    elif(isMaleNow):
        change_cols[col[0]] = 'male_'+col[1]['name'].lower().replace(' ', '_')
    else:
        change_cols[col[0]] = 'female_'+col[1]['name'].lower().replace(' ', '_')
        
sex_by_age_df.rename(columns=change_cols, inplace=True)

# Drop error and name columns
error_cols = ['name']
for col in sex_by_age_df.columns:
    if(col[0] == 'B'):
        error_cols.append(col)

sex_by_age_df.drop(columns=error_cols, inplace=True)

# Alter geoid so that it can merge with tract data
for i, row in sex_by_age_df.iterrows():
    sex_by_age_df.loc[i, 'geoid'] = row['geoid'][7:]
sex_by_age_df['geoid'] = pd.to_numeric(sex_by_age_df['geoid'])

In [8]:
sex_by_age_df.head(2)

,geoid,total_population,total_male_population,male_under_5_years,male_5_to_9_years,male_10_to_14_years,male_15_to_17_years,male_18_and_19_years,male_20_years,male_21_years,...,female_50_to_54_years,female_55_to_59_years,female_60_and_61_years,female_62_to_64_years,female_65_and_66_years,female_67_to_69_years,female_70_to_74_years,female_75_to_79_years,female_80_to_84_years,female_85_years_and_over
0,51001090100,2919,1354,78,67,96,7,25,9,0,...,129,143,82,84,70,86,83,103,56,38
1,51001090200,6407,3157,251,195,111,60,0,0,8,...,250,195,69,215,210,53,200,88,48,100


In [9]:
# Put sex_by_age data into census tract dataframe
tract_df = pd.merge(tract_df, sex_by_age_df, on='geoid', how='left')

- **Household income data processing**

In [10]:
household_path = '../dataset/household_income/'

# Process data.csv
with open(os.path.join(household_path, 'data.csv'), 'r') as infile:
    household_df = pd.read_csv(infile)

# Drop US and Virginia rows
household_df.drop(index=[0, 1], inplace=True)
household_df.reset_index(inplace=True, drop=True)

# Get column names from metadata.json
with open(os.path.join(household_path, 'metadata.json'), 'r') as infile:
    metadata = json.load(infile)

change_cols = {}
for col in metadata['tables']['B19001']['columns'].items():
    if(col[1]['name'] == 'Total:'):
        change_cols[col[0]] = 'total_household_income'
    else:
        change_cols[col[0]] = 'house_'+col[1]['name'].lower().replace(' ', '_').replace('$', '')
        
household_df.rename(columns=change_cols, inplace=True)

# Drop error and name columns
error_cols = ['name']
for col in household_df.columns:
    if(col[0] == 'B'):
        error_cols.append(col)

household_df.drop(columns=error_cols, inplace=True)

# Alter geoid so that it can merge with tract data
for i, row in household_df.iterrows():
    household_df.loc[i, 'geoid'] = row['geoid'][7:]
household_df['geoid'] = pd.to_numeric(household_df['geoid'])

In [11]:
household_df.head(2)

,geoid,total_household_income,"house_less_than_10,000","house_10,000_to_14,999","house_15,000_to_19,999","house_20,000_to_24,999","house_25,000_to_29,999","house_30,000_to_34,999","house_35,000_to_39,999","house_40,000_to_44,999","house_45,000_to_49,999","house_50,000_to_59,999","house_60,000_to_74,999","house_75,000_to_99,999","house_100,000_to_124,999","house_125,000_to_149,999","house_150,000_to_199,999","house_200,000_or_more"
0,51001090100,1382,120,41,77,79,104,92,65,85,74,108,172,145,77,64,23,56
1,51001090200,2638,177,115,216,279,198,151,223,68,234,206,275,266,122,18,15,75


In [12]:
# Put household_income data into census tract dataframe
tract_df = pd.merge(tract_df, household_df, on='geoid', how='left')

- **Age by language data processing**

In [13]:
language_path = '../dataset/age_by_language/'

# Process data.csv
with open(os.path.join(language_path, 'data.csv'), 'r') as infile:
    language_df = pd.read_csv(infile)

# Drop US and Virginia rows
language_df.drop(index=[0, 1], inplace=True)
language_df.reset_index(inplace=True, drop=True)

# Get column names from metadata.json
with open(os.path.join(language_path, 'metadata.json'), 'r') as infile:
    metadata = json.load(infile)

prefix = ""
change_cols = {}
for col in metadata['tables']['B16007']['columns'].items():
    if(col[1]['name'] == 'Total:'):
        change_cols[col[0]] = 'total'
    elif(col[1]['name'] == '5 to 17 years:'):
        change_cols[col[0]] = 'total_5_to_17'
        prefix = '5_to_17_'
    elif(col[1]['name'] == '18 to 64 years:'):
        change_cols[col[0]] = 'total_18_to_64'
        prefix = '18_to_64_'
    else:
        change_cols[col[0]] = prefix+col[1]['name'].lower().replace(' ', '_')
        
language_df.rename(columns=change_cols, inplace=True)

# Drop error, name and total columns
error_cols = ['name', 'total']
for col in language_df.columns:
    if(col[0] == 'B'):
        error_cols.append(col)

language_df.drop(columns=error_cols, inplace=True)

# Alter geoid so that it can merge with tract data
for i, row in language_df.iterrows():
    language_df.loc[i, 'geoid'] = row['geoid'][7:]
language_df['geoid'] = pd.to_numeric(language_df['geoid'])

In [14]:
language_df.head(2)

,geoid,total_5_to_17,5_to_17_speak_only_english,5_to_17_speak_spanish,5_to_17_speak_other_indo-european_languages,5_to_17_speak_asian_and_pacific_island_languages,5_to_17_speak_other_languages,total_18_to_64,18_to_64_speak_only_english,18_to_64_speak_spanish,18_to_64_speak_other_indo-european_languages,18_to_64_speak_asian_and_pacific_island_languages,18_to_64_speak_other_languages,18_to_64_65_years_and_over:,18_to_64_speak_only_english,18_to_64_speak_spanish,18_to_64_speak_other_indo-european_languages,18_to_64_speak_asian_and_pacific_island_languages,18_to_64_speak_other_languages
0,51001090100,298,298,0,0,0,0,1656,1595,37,24,0,0,818,769,0,49,0,0
1,51001090200,869,762,61,46,0,0,3681,3421,237,23,0,0,1271,1247,24,0,0,0


In [15]:
# Put age_by_language data into census tract dataframe
tract_df = pd.merge(tract_df, language_df, on='geoid', how='left')

In [16]:
# Final tract_df
tract_df.head()

,usps,geoid,pop10,hu10,aland,awater,aland_sqmi,awater_sqmi,intptlat,intptlong,...,18_to_64_speak_spanish,18_to_64_speak_other_indo-european_languages,18_to_64_speak_asian_and_pacific_island_languages,18_to_64_speak_other_languages,18_to_64_65_years_and_over:,18_to_64_speak_only_english,18_to_64_speak_spanish,18_to_64_speak_other_indo-european_languages,18_to_64_speak_asian_and_pacific_island_languages,18_to_64_speak_other_languages
0,VA,51001090100,2941,4517,18529985,37927447,7.154,14.644,37.945138,-75.362044,...,37.0,24.0,0.0,0.0,818.0,769.0,0.0,49.0,0.0,0.0
1,VA,51001090200,6156,4259,187805809,36543634,72.512,14.110,37.953865,-75.503313,...,237.0,23.0,0.0,0.0,1271.0,1247.0,24.0,0.0,0.0,0.0
2,VA,51001090300,2335,1221,128431098,61781712,49.588,23.854,37.887621,-75.662907,...,58.0,3.0,23.0,0.0,508.0,507.0,0.0,1.0,0.0,0.0
3,VA,51001090400,6234,2687,193719276,35803197,74.795,13.824,37.752376,-75.598061,...,883.0,249.0,0.0,0.0,1043.0,1008.0,35.0,0.0,0.0,0.0
4,VA,51001090500,2849,1361,65940306,39496129,25.460,15.250,37.814134,-75.696946,...,172.0,122.0,7.0,0.0,449.0,449.0,0.0,0.0,0.0,0.0


In [17]:
tract_df.columns

Index(['usps', 'geoid', 'pop10', 'hu10', 'aland', 'awater', 'aland_sqmi',
       'awater_sqmi', 'intptlat',
       'intptlong                                                                                                                  ',
       'total_population', 'total_male_population', 'male_under_5_years',
       'male_5_to_9_years', 'male_10_to_14_years', 'male_15_to_17_years',
       'male_18_and_19_years', 'male_20_years', 'male_21_years',
       'male_22_to_24_years', 'male_25_to_29_years', 'male_30_to_34_years',
       'male_35_to_39_years', 'male_40_to_44_years', 'male_45_to_49_years',
       'male_50_to_54_years', 'male_55_to_59_years', 'male_60_and_61_years',
       'male_62_to_64_years', 'male_65_and_66_years', 'male_67_to_69_years',
       'male_70_to_74_years', 'male_75_to_79_years', 'male_80_to_84_years',
       'male_85_years_and_over', 'total_female_population',
       'female_under_5_years', 'female_5_to_9_years', 'female_10_to_14_years',
       'female_15_to_17_

## 3. Add new features to coordinates dataset
This section includes:
1. For each business, decide which census tract it belongs to.
2. Add corresponding census tract features.

To calculate the distance between two spots on the earth, we here adopted the Haversine formula. The code is based on [Haversine formula in Python](https://stackoverflow.com/a/4913653/8163369) on StackOverFlow.

In [18]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance (km) between two points 
    on the earth (specified in decimal degrees).
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return abs(c * r)

In [19]:
def get_tract_index(tract_df, lat, lon):
    """For coordinate (lat, lon), decide which census tract it belongs to.
    """
    min_dist = 10000
    tract_index = -1
    for i, row in tract_df.iterrows():
        print(row['intptlong'])
        break
        dist = haversine(lon, lat, row['intptlong'], row['intptlat'])
        if(dist < min_dist):
            min_dist = dist
            tract_index = i
    return tract_index

In [20]:
for i, row in origin_data.iterrows():
    lat, lon = row[['latitude', 'longitude']]
    print(get_tract_index(tract_df, lat, lon))
    
    break

KeyError: 'intptlong'

## 2. Get Geographic Data
GeoProcessor is a class used for processing geographic features (latitude and longitude) in a dataset.

The basic idea is that, first build a geographic price model based on given data, and then provide a method which can give price evaluation to any specific geographic spot.

In [6]:
class GeoProcessor:
    
    def __init__(self, lon1, lat1, lon2, lat2, unit_len=0.1):
        """Initialize a geographic model.
        
        Initialize a geographic model, whose range is based 
        on given latitudes and longitudes.
        
        (lon1, lat1)-----------(lon2, lat1)
        |                                 |
        |h                                |
        |                l                |
        (lon1, lat2)-----------(lon2, lat2)
        
        Args:
            lat1: The upper latitude.
            lat2: The lower latitude.
            lon1: The left longitude.
            lon2: The right longitude.
            unit_len: The unit length in price map, i.e. the
                distance difference between two indices in
                the price map.
        """
        self._lon1, self._lat1 = lon1, lat1
        self._lon2, self._lat2 = lon2, lat2
        self._unit = unit_len
        
        # Calculate the size of map (kilometer)
        h = haversine(lon1, lat1, lon1, lat2)
        l = haversine(lon1, lat1, lon2, lat1),
        self._price_map = np.zeros([int(h/unit_len) + 1,
                                   int(l/unit_len) + 1])
        self._weight_map = np.zeros([int(h/unit_len) + 1,
                                   int(l/unit_len) + 1])
    
    def fit(X, y, lon_idx=0, lat_idx=1, helper_idx=-1,
           att_coef=0.5):
        """Train price map based on dataset X.
        
        Args:
            X: m*n 2d array represents the dataset, where m
                is the number of samples, n is the number of
                features.
            y: The target classification labels.
            lon_idx: The longitude feature column index in X.
                Default is 0.
            lat_idx: The latitude feature column index in X.
                Default is 1.
            helper_idx: The feature which measures the
                importance of samples. -1 means there is no
                such a helper feature. Default is -1.
            att_coef: The attenuation coefficient. It will
                control decreasing the impacts of a sample
                to its neighborhood. e.g. A sample will have
                higher impacts to nearer positions and lower
                impacts to farer positions.
        """
        for k in range(len(X)):
            sample = X[k, :]
            
            # Get indices of current sample
            i, j = ll2idx(sample[lat_idx], sample[lon_idx])
            
            # Get weight of current sample
            if helper_idx == -1:
                weight = 1
            else:
                weight = sample[helper_idx]
            
            # Using DFS to update price and weight map
            
    
    def ll2idx(lat, lon):
        """Transform coordinates to indices in price map.
        
        Args:
            lat: The latitude.
            lon: The longitude.
            
        Returns:
            i: The first index.
            j: The second index.
        """
        if(lat > self._lat1 or lat < self._lat2
           or lon < self._lon1 or lon > self._lon2):
            print("Invalid coordinates.")
            return [-1, -1]
        
        h = harversine(self._lon1, self._lat1, 
                       self._lon1, lat)
        l = harversine(self._lon1, self._lat1,
                       lon, self._lat1)
        return [int(h/self._unit), int(l/self._unit)]
    
    def haversine(lon1, lat1, lon2, lat2):
        """
        Calculate the great circle distance between two points 
        on the earth (specified in decimal degrees).
        """
        # convert decimal degrees to radians 
        lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

        # haversine formula 
        dlon = lon2 - lon1 
        dlat = lat2 - lat1 
        a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
        c = 2 * asin(sqrt(a)) 
        r = 6371 # Radius of earth in kilometers. Use 3956 for miles
        return abs(c * r)

## 3. Generate dataset for Gephi

Now we are going to consider the relation between restaurants as a undirected network graph. Definition of this graph is listed as follows:

**Node:** Each node represents a business, which can be uniquely represented by its id.

**Edge:** Each edge $(u, v)$ represents that the business $u$ is a neighbor of $v$ $(u \neq v)$. Here, we define a parameter, neighbor_dist, as the maximum distance (kilometer) between two neighbors. If the distance of two businesses, $u$ and $v$, is less than neighbor_dist, then we will have edge $(u, v)$ and $(v, u)$.

To calculate the distance between two businesses on the earth, we here adopted the Haversine formula. The code is based on [Haversine formula in Python](https://stackoverflow.com/a/4913653/8163369) on StackOverFlow.

In [22]:
neighbor_dist = 0.5 # The maximum distance (kilometer) between two neighbors

lats, lons = np.array(data['latitude']), np.array(data['longitude']) # latitude and longitude of businesses
num_samples = data.shape[0] # total number of businesses

edges = []

for i in range(0, num_samples):
    for j in range(i + 1, num_samples):
        dist = haversine(lons[i], lats[i], lons[j], lats[j])
        if(dist < neighbor_dist):
            edges.append([i, j, dist + 0.001])
            edges.append([j, i, dist + 0.001])

In [23]:
len(edges)

21978

**Output nodes and as .csv file:** for use of Gephi.

In [24]:
# Generate edges and nodes dataframe as the requirement of Gephi
edges_df = pd.DataFrame(edges, columns=['Source', 'Target', 'Weight'])

nodes_df = data.rename(columns={'id': 'label'})
nodes_df = nodes_df.assign(id=range(0, num_samples))

display(edges_df.head())
display(nodes_df.head())

,Source,Target,Weight
0,12,13,0.181609
1,13,12,0.181609
2,14,17,0.284748
3,17,14,0.284748
4,14,18,0.172542


,label,name,latitude,longitude,rating,review_count,price,id
0,bwCj2AcoOroZfCTxb6rCcg,A Better Burger,36.689639,-83.108766,3.5,6,2,0
1,S9S9kFJSkmfpbjFForCWLQ,El Castillo,36.726337,-83.099858,4.0,2,1,1
2,np8uV1xll22Yr-Q-B-ImkA,Rooster's Pub,36.758436,-83.027057,4.5,4,1,2
3,HGY1ojoLu07P_ky2LeRguQ,Redstone Restaurant,36.689259,-82.753040,4.5,3,1,3
4,J5XS3VmxnLKhNlpiwDJ-3A,Little Mexico,36.859367,-82.756744,4.0,5,1,4


In [25]:
# Write dataframes into files
nodes_file = './Gephi/coordinates_nodes.csv'
edges_file = './Gephi/coordinates_edges.csv'

with open(nodes_file, 'w', encoding='utf-8') as outFile:
    nodes_df.to_csv(outFile, sep=',', encoding='utf-8', index=False)

with open(edges_file, 'w') as outFile:
    edges_df.to_csv(outFile, sep=',', encoding='utf-8', index=False)